# Azure AI Search with Azure AI Vision multimodal embeddings for text-to-image queries

As a scenario, this code shows you an approach for text-to-image vector queries. As a technical sample, it demonstrates how to call a custom embedding model for situations where you want models other an Azure OpenAI or OpenAI for vectorization. The multimodal embeddings used in this sample are provided by [Azure AI Vision 4.0](https://learn.microsoft.com/azure/ai-services/computer-vision/how-to/image-retrieval) and the [Image Retrieval REST API](https://learn.microsoft.com/rest/api/computervision/image-retrieval) which supports built-in vectorization of images. 

For indexing, the pattern uses a custom skill to wrap an Azure function app used to call the Image Retrieval API. Provisioning of this function app and custom skill is fully automated and included as a step in this notebook.

The function app is also used during queries, as the vectorizer. A vectorizer specifies which embedding model to use for vectorizing a text query string. As always, it's strongly recommended that query vectorization is performed using the same embedding model used for document vectorization during indexing.

### Prerequisites

+ [Azure AI Search](https://learn.microsoft.com/azure/search/search-create-service-portal), any region and tier, but we recommend Basic or higher for this workload.

+ [Azure Blob storage](https://learn.microsoft.com/azure/storage/common/storage-account-create), used as the data source during indexing.

+ [azd](https://learn.microsoft.com/azure/developer/azure-developer-cli/install-azd), used to deploy an Azure function app and Azure AI Vision in your Azure subscription.

We use the [Azure Python SDK](https://learn.microsoft.com/en-us/python/api/azure-search-documents/?view=azure-python-preview) for indexer-driven indexing and vector query operations.

### Set up a Python virtual environment in Visual Studio Code

1. Open the Command Palette (Ctrl+Shift+P).
1. Search for **Python: Create Environment**.
1. Select **Venv**.
1. Select a Python interpreter. Choose 3.10 or later.

It can take a minute to set up. If you run into problems, see [Python environments in VS Code](https://code.visualstudio.com/docs/python/environments).

In [ ]:
! pip install -r azure-search-vector-image-python-sample-requirements.txt

In [3]:
from dotenv import load_dotenv
from azure.identity import DefaultAzureCredential
from azure.core.credentials import AzureKeyCredential
import os

load_dotenv() # take environment variables from .env.

# Variables not used here do not need to be updated in your .env file
endpoint = os.environ["AZURE_SEARCH_SERVICE_ENDPOINT"]
credential = AzureKeyCredential(os.environ["AZURE_SEARCH_ADMIN_KEY"]) if len(os.environ["AZURE_SEARCH_ADMIN_KEY"]) > 0 else DefaultAzureCredential()
index_name = os.environ["AZURE_SEARCH_INDEX"]
blob_connection_string = os.environ["BLOB_CONNECTION_STRING"]
blob_container_name = os.environ["BLOB_CONTAINER_NAME"]
function_app_url = os.environ["function_app_url"]
sas_token = os.environ["SAS_TOKEN"]

In [4]:
from azure.search.documents.indexes import SearchIndexerClient
from azure.search.documents.indexes.models import SearchIndexerDataContainer, SearchIndexerDataSourceConnection
# Create a data source 
ds_client = SearchIndexerClient(endpoint, credential)
container = SearchIndexerDataContainer(name=blob_container_name)
data_source_connection = SearchIndexerDataSourceConnection(
    name=f"{index_name}-blob",
    type="azureblob",
    connection_string=blob_connection_string,
    container=container
)
data_source = ds_client.create_or_update_data_source_connection(data_source_connection)

print(f"Data source '{data_source.name}' created or updated")

Data source 'image-index-blob' created or updated


In [5]:
#create skillset
from azure.search.documents.indexes.models import (
    WebApiSkill,
    InputFieldMappingEntry,
    OutputFieldMappingEntry,
    SearchIndexerSkillset
)

# Create a skillset  
skillset_name = f"{index_name}-skillset"  
  
skill = WebApiSkill(  
    description="Skill to generate image embeddings via a custom endpoint",  
    context="/document",
    http_method="POST",
    batch_size=10, # Controls how many images are sent to the custom skill at a time
    uri=function_app_url,
    inputs=[
        InputFieldMappingEntry(name="imageUrl", source="/document/metadata_storage_path"),
        InputFieldMappingEntry(name="sasToken", source="/document/metadata_storage_sas_token"),  
    ],  
    outputs=[  
        OutputFieldMappingEntry(name="vector", target_name="vector")
    ],
)
  
skillset = SearchIndexerSkillset(  
    name=skillset_name,  
    description="Skillset to extract image vector",  
    skills=[skill],  
)
  
ds_client.create_or_update_skillset(skillset)  
print(f'Skillset {skillset.name} created')  

Skillset image-index-skillset created


In [6]:
#create index
from azure.search.documents.indexes import SearchIndexClient
from azure.search.documents.indexes.models import (
    SimpleField,
    SearchFieldDataType,
    SearchField,
    VectorSearch,
    HnswAlgorithmConfiguration,
    VectorSearchProfile,
    SearchIndex,
    CustomVectorizer,
    CustomWebApiParameters
)

# Create a search index  
index_client = SearchIndexClient(endpoint=endpoint, credential=credential)  
fields = [  
    SimpleField(name="id", type=SearchFieldDataType.String, key=True, sortable=True, filterable=True, facetable=True),  
    SearchField(name="imageUrl", type=SearchFieldDataType.String),  
    SearchField(name="title", type=SearchFieldDataType.String),  
    SearchField(  
        name="imageVector",  
        type=SearchFieldDataType.Collection(SearchFieldDataType.Single),  
        vector_search_dimensions=1024,  
        vector_search_profile_name="myHnswProfile",  
    ),  
]  
  
# Configure the vector search configuration  
vector_search = VectorSearch(  
    algorithms=[  
        HnswAlgorithmConfiguration(  
            name="myHnsw"
        )
    ],  
   profiles=[  
        VectorSearchProfile(  
            name="myHnswProfile",  
            algorithm_configuration_name="myHnsw", 
            vectorizer="customVectorizer"
        )
    ],
    vectorizers=[  
        CustomVectorizer(name="customVectorizer", custom_web_api_parameters=CustomWebApiParameters(uri=function_app_url))
    ]
)
  
# Create the search index with the vector search configuration  
index = SearchIndex(name=index_name, fields=fields, vector_search=vector_search)  
result = index_client.create_or_update_index(index)  
print(f"{result.name} created")  

image-index created


In [7]:
#create indexer
from azure.search.documents.indexes.models import (
    SearchIndexer,
    FieldMapping
)

# Create an indexer  
indexer_name = f"{index_name}-indexer"  
indexer = SearchIndexer(  
    name=indexer_name,  
    description="Indexer to process images",  
    skillset_name=skillset_name,  
    target_index_name=index_name,  
    data_source_name=data_source.name,  
    field_mappings=[  
        FieldMapping(source_field_name="metadata_storage_path", target_field_name="imageUrl"),  
        FieldMapping(source_field_name="metadata_storage_name", target_field_name="title")  
    ],  
    output_field_mappings=[  
        FieldMapping(source_field_name="/document/vector", target_field_name="imageVector")  
    ]  
)  
  
indexer_client = SearchIndexerClient(endpoint, credential)  
indexer_result = indexer_client.create_or_update_indexer(indexer)  
  
# Run the indexer  
indexer_client.run_indexer(indexer_name)  
print(f'{indexer_name} is created and running. It will be several minutes before you can run the queries.')

image-index-indexer is created and running. It will be several minutes before you can run the queries.


In [8]:
from azure.search.documents import SearchClient
from azure.search.documents.models import (
    VectorizableTextQuery
)
from IPython.display import Image


query = "red apple"  
  
# Initialize the SearchClient  
search_client = SearchClient(endpoint, index_name, credential)  
vector_query = VectorizableTextQuery(text=query, k_nearest_neighbors=1, fields="imageVector")  

# Perform vector search  
results = search_client.search(  
    search_text=None,  
    vector_queries= [vector_query],
    select=["title", "imageUrl"],
    top=1
)   
  
# Print the search results  
for result in results:  
    print(f"Title: {result['title']}")  
    print(f"Image URL: {result['imageUrl']}") 
    display(Image(url=result['imageUrl'] + "?" + sas_token)) 
    print("\n") 


Title: RedDelicious.jpeg
Image URL: https://mmxxblobxstore.blob.core.windows.net/images/RedDelicious.jpeg


In [9]:
import requests

# Generate image embeddings for the query (for context, this is a photo of a red apple)
query = "https://upload.wikimedia.org/wikipedia/commons/a/a6/Pink_lady_and_cross_section.jpg"

response = requests.post(function_app_url, json={ "values": [ { "recordId": 0, "data": { "imageUrl": query, "sasToken": "" } } ] })  
response.raise_for_status()
vector = response.json()["values"][0]["data"]["vector"]

print(vector)

[-3.6191406, 0.5019531, 0.46118164, 1.4804688, 1.7939453, -1.0195312, 0.89404297, 3.390625, -5.1953125, 3.4628906, 1.7089844, 2.5664062, -4.78125, -0.8652344, 0.37524414, 1.4404297, 1.3808594, 0.89208984, -3.59375, 1.9023438, -1.9042969, 2.09375, -0.41601562, -4.2695312, 4.5585938, 0.6425781, 3.8242188, -1.4453125, -2.0683594, 0.10473633, 2.0976562, -3.5683594, -0.5390625, 1.3496094, -0.06518555, -0.31201172, 1.2890625, -0.09338379, -0.80615234, -1.3730469, -2.8710938, -2.4199219, -2.8457031, -0.4189453, -0.24108887, 0.4260254, -0.81884766, 1.3583984, 0.3630371, 3.8242188, -2.3398438, -0.953125, -1.0527344, -0.012001038, 0.01272583, 0.73535156, -3.7949219, -1.6220703, 0.33935547, 4.4609375, -1.6601562, -1.4160156, -3.0527344, 1.5732422, 2.46875, -1.8134766, 1.7890625, 1.8496094, -0.9707031, -0.23547363, 0.06304932, -0.3100586, -1.6064453, -0.9453125, -2.4453125, -1.0917969, -1.3984375, -2.1933594, -2.3789062, -0.24719238, -1.9423828, 0.5205078, -5.2304688, -1.484375, -0.5385742, 2.8203

In [10]:
from azure.search.documents.models import VectorizedQuery

# Initialize the SearchClient  
search_client = SearchClient(endpoint, index_name, credential)  
vector_query = VectorizedQuery(vector=vector, k_nearest_neighbors=1, fields="imageVector")  

# Perform vector search  
results = search_client.search(  
    search_text=None,  
    vector_queries= [vector_query],
    select=["title", "imageUrl"],
    top=1
)   

print("Source Image")
display(Image(url=query))
# Print the search results  
for result in results:  
    print(f"Title: {result['title']}")  
    print(f"Image URL: {result['imageUrl']}") 
    display(Image(url=result['imageUrl'] + "?" + sas_token))
    #print("\n") 

Source Image


Title: Honeycrisp.jpeg
Image URL: https://mmxxblobxstore.blob.core.windows.net/images/Honeycrisp.jpeg
